In [1]:
import pandas as pd

# Прочитаем данные
df = pd.read_parquet('transaction_fraud_data.parquet', engine='pyarrow')

# Доля мошеннических транзакций
fraud_share = df['is_fraud'].mean()
print(f'Доля мошеннических транзакций: {fraud_share:.4f}')

Доля мошеннических транзакций: 0.1997


In [2]:
# Топ-5 стран по количеству мошеннических транзакций
top_countries = df[df['is_fraud'] == True]['country'].value_counts().head(5).index.tolist()
result = ','.join(top_countries)
print(result)

Russia,Mexico,Brazil,Nigeria,Australia


In [3]:
# Доля мошенничества среди транзакций у продавцов с высоким риском
high_risk_fraud_share = df[df['is_high_risk_vendor'] == True]['is_fraud'].mean()
print(f'Доля мошенничества у high risk vendor: {high_risk_fraud_share:.4f}')

Доля мошенничества у high risk vendor: 0.2000


In [4]:
# Город с наибольшим средним чеком по операциям fast_food (без Unknown City)
fast_food_df = df[(df['vendor_type'].str.lower() == 'fast_food') & (df['city'] != 'Unknown City')]
city_with_max_avg_fast_food = fast_food_df.groupby('city')['amount'].mean().idxmax()
print(city_with_max_avg_fast_food)

Chicago


In [5]:
# Средний чек для всех немошеннических операций в долларах США
exchange = pd.read_parquet('historical_currency_exchange.parquet')
non_fraud_df = df[df['is_fraud'] == False].copy()
non_fraud_df['date'] = pd.to_datetime(non_fraud_df['timestamp']).dt.date
exchange['date'] = pd.to_datetime(exchange['date']).dt.date
merged = pd.merge(non_fraud_df, exchange, left_on=['date'], right_on=['date'], how='left')
def convert(row):
    rate = row.get(row['currency'], None)
    return row['amount'] / rate if rate and rate > 0 else None
merged['amount_usd'] = merged.apply(convert, axis=1)
mean_usd = merged['amount_usd'].mean()
print(f'Средний чек немошеннических операций в USD: {mean_usd:.2f}')

Средний чек немошеннических операций в USD: 459.78


In [6]:
# Стандартное отклонение для всех немошеннических операций в долларах США
std_usd = merged['amount_usd'].std()
print(f'Стандартное отклонение немошеннических операций в USD: {std_usd:.2f}')

Стандартное отклонение немошеннических операций в USD: 417.01


In [7]:
# Средний чек для всех мошеннических операций в долларах США
fraud_df = df[df['is_fraud'] == True].copy()
fraud_df['date'] = pd.to_datetime(fraud_df['timestamp']).dt.date
merged_fraud = pd.merge(fraud_df, exchange, left_on=['date'], right_on=['date'], how='left')
def convert_fraud(row):
    rate = row.get(row['currency'], None)
    return row['amount'] / rate if rate and rate > 0 else None
merged_fraud['amount_usd'] = merged_fraud.apply(convert_fraud, axis=1)
mean_fraud_usd = merged_fraud['amount_usd'].mean()
print(f'Средний чек мошеннических операций в USD: {mean_fraud_usd:.2f}')

Средний чек мошеннических операций в USD: 874.61


In [8]:
# Стандартное отклонение для всех мошеннических операций в долларах США
std_fraud_usd = merged_fraud['amount_usd'].std()
print(f'Стандартное отклонение мошеннических операций в USD: {std_fraud_usd:.2f}')

Стандартное отклонение мошеннических операций в USD: 1349.88


In [9]:
# Количество клиентов с медианой уникальных продавцов выше 95-го квантиля
unique_merchants = df[['customer_id', 'last_hour_activity']].copy()
unique_merchants['unique_merchants'] = unique_merchants['last_hour_activity'].apply(lambda x: x['unique_merchants'])
medians = unique_merchants.groupby('customer_id')['unique_merchants'].median()
q95 = medians.quantile(0.95)
dangerous_clients = (medians > q95).sum()
print(dangerous_clients)

229


In [10]:
# Количество уникальных (distinct) пар card_number и customer_id и всего совпадений
distinct_pairs = df[['card_number', 'customer_id']].drop_duplicates().shape[0]
total_pairs = df.shape[0]
print(f'Уникальных (distinct) пар: {distinct_pairs}')
print(f'Всего совпадений: {total_pairs}')

Уникальных (distinct) пар: 5000
Всего совпадений: 7483766


In [11]:
# Топ-5 card_number для customer_id = CUST_10715
top_cards = df[df['customer_id'] == 'CUST_10715']['card_number'].value_counts().head(5).index.tolist()
print(top_cards)

[5251909460951913]


In [12]:
# Количество вхождений customer_id = CUST_10715
count_cust_10715 = (df['customer_id'] == 'CUST_10715').sum()
print(count_cust_10715)

1721


In [13]:
# Время всех записей для customer_id = CUST_10715
times = df[df['customer_id'] == 'CUST_10715']['timestamp'].tolist()
print(times)

[Timestamp('2024-09-30 00:00:02.273762'), Timestamp('2024-09-30 03:08:54.549083'), Timestamp('2024-09-30 03:45:10.451094'), Timestamp('2024-09-30 03:55:46.263634'), Timestamp('2024-09-30 04:03:02.436919'), Timestamp('2024-09-30 05:36:29.285764'), Timestamp('2024-09-30 06:16:31.268111'), Timestamp('2024-09-30 07:29:24.271802'), Timestamp('2024-09-30 07:39:14.346950'), Timestamp('2024-09-30 08:23:04.419481'), Timestamp('2024-09-30 08:28:41.262022'), Timestamp('2024-09-30 08:39:12.262298'), Timestamp('2024-09-30 08:41:29.534495'), Timestamp('2024-09-30 08:42:41.380408'), Timestamp('2024-09-30 09:13:47.397979'), Timestamp('2024-09-30 09:31:05.484031'), Timestamp('2024-09-30 09:52:48.269750'), Timestamp('2024-09-30 11:09:19.260273'), Timestamp('2024-09-30 11:49:22.318779'), Timestamp('2024-09-30 12:25:27.561529'), Timestamp('2024-09-30 12:31:35.544147'), Timestamp('2024-09-30 13:24:05.456467'), Timestamp('2024-09-30 13:42:14.564857'), Timestamp('2024-09-30 13:45:47.382429'), Timestamp('2024

In [14]:
# Показатели last_hour_activity по указанным timestamp для customer_id = CUST_10715
import pandas as pd
cust_df = df[df['customer_id'] == 'CUST_10715'].copy()
cust_df['timestamp'] = pd.to_datetime(cust_df['timestamp'])
target_times = [pd.Timestamp('2024-09-30 03:08:54.549083'), pd.Timestamp('2024-09-30 03:45:10.451094'), pd.Timestamp('2024-09-30 03:55:46.263634')]
for t in target_times:
    row = cust_df[cust_df['timestamp'] == t]
    if not row.empty:
        amount = row.iloc[0]['amount']
        is_fraud = row.iloc[0]['is_fraud']
        print(f"{t}: amount={amount} is_fraud={is_fraud}")
    else:
        print(f"{t}: not found")

# last_hour_activity для Timestamp('2024-09-30 04:03:02.436919')
t2 = pd.Timestamp('2024-09-30 04:03:02.436919')
row2 = cust_df[cust_df['timestamp'] == t2]
if not row2.empty:
    amount = row2.iloc[0]['amount']
    print(f"{t}: amount={amount}")
    print(f"last_hour_activity for {t2}: {row2.iloc[0]['last_hour_activity']}")
else:
    print(f"{t2}: not found")

2024-09-30 03:08:54.549083: amount=324.54 is_fraud=True
2024-09-30 03:45:10.451094: amount=51257.68 is_fraud=False
2024-09-30 03:55:46.263634: amount=154826.16 is_fraud=True
2024-09-30 03:55:46.263634: amount=9767.24
last_hour_activity for 2024-09-30 04:03:02.436919: {'num_transactions': 1239, 'total_amount': 135773867.54822382, 'unique_merchants': 105, 'unique_countries': 12, 'max_single_amount': 3288900.3754569697}


In [15]:
# Операции с amount > 3288900 и < 3288901: вывод клиента, времени, суммы и is_fraud
high_amount_df = df[(df['amount'] > 3288900) & (df['amount'] < 3288901)][['customer_id', 'timestamp', 'amount', 'is_fraud']]
print(high_amount_df)

        customer_id                  timestamp      amount  is_fraud
4815488  CUST_10715 2024-10-19 22:43:25.282272  3288900.38      True


In [16]:
# Количество уникальных total_amount и max_single_amount в last_hour_activity для CUST_10715
cust_df = df[df['customer_id'] == 'CUST_10715'].copy()
total_amounts = cust_df['last_hour_activity'].apply(lambda x: x['total_amount'])
max_single_amounts = cust_df['last_hour_activity'].apply(lambda x: x['max_single_amount'])
unique_total_amounts = total_amounts.nunique()
unique_max_single_amounts = max_single_amounts.nunique()
print(f'Уникальных total_amount: {unique_total_amounts}')
print(f'Уникальных max_single_amount: {unique_max_single_amounts}')

Уникальных total_amount: 1721
Уникальных max_single_amount: 36


In [17]:
# Изменение total_amount и вывод amount между указанными timestamp для CUST_10715

cust_df = df[df['customer_id'] == 'CUST_10715'].copy()
cust_df['timestamp'] = pd.to_datetime(cust_df['timestamp'])
target_times = [pd.Timestamp('2024-09-30 03:08:54.549083'), pd.Timestamp('2024-09-30 03:45:10.451094'), pd.Timestamp('2024-09-30 03:55:46.263634')]
total_amounts = []
for t in target_times:
    row = cust_df[cust_df['timestamp'] == t]
    if not row.empty:
        total_amount = row.iloc[0]['last_hour_activity']['total_amount']
        amount = row.iloc[0]['amount']
        total_amounts.append((t, total_amount, amount))
        print(f"{t}: total_amount={total_amount}, amount={amount}")
    else:
        print(f"{t}: not found")
if len(total_amounts) > 1:
    for i in range(1, len(total_amounts)):
        diff = total_amounts[i][1] - total_amounts[i-1][1]
        print(f"Change from {total_amounts[i-1][0]} to {total_amounts[i][0]}: {diff}")

2024-09-30 03:08:54.549083: total_amount=173709175.71044284, amount=324.54
2024-09-30 03:45:10.451094: total_amount=140645155.28502682, amount=51257.68
2024-09-30 03:55:46.263634: total_amount=20281500.604026493, amount=154826.16
Change from 2024-09-30 03:08:54.549083 to 2024-09-30 03:45:10.451094: -33064020.425416023
Change from 2024-09-30 03:45:10.451094 to 2024-09-30 03:55:46.263634: -120363654.68100032


In [18]:
import numpy as np

# Фильтруем по customer_id
cust_df = df[df['customer_id'] == 'CUST_10715'].copy()

# Извлекаем total_amount из last_hour_activity
cust_df['total_amount'] = cust_df['last_hour_activity'].apply(lambda x: x['total_amount'] if isinstance(x, dict) and 'total_amount' in x else np.nan)

# Сортируем по времени
cust_df = cust_df.sort_values('timestamp').reset_index(drop=True)

# Ищем пары строк, где разница по модулю между total_amount в нужном диапазоне
found = False
for i in range(len(cust_df)):
    for j in range(i+1, len(cust_df)):
        diff = abs(cust_df.loc[i, 'total_amount'] - cust_df.loc[j, 'total_amount'])
        if 3288900 <= diff <= 3288901:
            print(f"Пара индексов: {i}, {j}")
            print(f"Времена: {cust_df.loc[i, 'timestamp']}, {cust_df.loc[j, 'timestamp']}")
            print(f"total_amount: {cust_df.loc[i, 'total_amount']}, {cust_df.loc[j, 'total_amount']}")
            print(f"Разница по модулю: {diff}")
            found = True
if not found:
    print("Нет таких пар строк.")

Нет таких пар строк.
